<a href="https://colab.research.google.com/github/HromovAndrey/II_mood_6_part_2_practic/blob/main/%D0%9A%D0%BE%D0%BF%D0%B8%D1%8F_%D0%B1%D0%BB%D0%BE%D0%BA%D0%BD%D0%BE%D1%82%D0%B0_%22%D0%9F%D1%80%D0%B0%D0%BA%D1%82%D0%B8%D0%BA%D0%B0_text_classification_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Завдання

[Посилання](https://www.kaggle.com/datasets/banuprakashv/news-articles-classification-dataset-for-nlp-and-ml/data)

За допомогою `pytorch` проведіть класифікацію тексту:

* Завантажте усі датафрейми
* Добавте до кожного стовпчик `labels` з відповідним йому класом(bussiness, sports, education, etc). назви класів беріть з назв файлів
* Об'єднайте всі датафрейми в один, див [pd.concat](https://pandas.pydata.org/docs/reference/api/pandas.concat.html)
* Створіть стовпчик `text` в якому зберігатиметься текст для подальшої роботи. Можети вибрати такі варіанти:
  * Взяти один зі стовпчиків `headline`, `description`, `content`
  * Об'єднати тексти зі стовпчиків `headline`, `description`, `content`
* Видаліть стовпчики `headline`, `description`, `content`, `url`
* Створіть датасет з використанням `Glove`
* Перевірте розмірність даних у датасеті
* Розділіть дані на тренувальні та тестові, створіть `data_loader`
* Створіть модель та натренуйте її
* Виведіть результати класифікації

In [ ]:
!pip install torch==2.0.1 torchtext==0.15.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 42.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 90.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 81.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 47.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.4/168.4 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.6/102.6 MB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.2/173.2 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.1/177.1 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98

In [13]:
import pandas as pd
import zipfile
import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import Dataset, DataLoader
from torchtext.vocab import GloVe
import torch.nn as nn
import torch.optim as optim

In [12]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [18]:

# Распаковка ZIP-файла
with zipfile.ZipFile("/content/archive.zip", 'r') as zip_ref:
    zip_ref.extractall("/content/data")  # Извлекаем все файлы в папку 'data'

# Проверка списка файлов в архиве
import os
files = os.listdir("/content/data")
print(files)  # Печатает список файлов: ['business_data.csv', 'education_data.csv', ...]

# Загрузка всех файлов в датафреймы и добавление метки (label)
dfs = []
for file_name in files:
    df = pd.read_csv(f"/content/data/{file_name}")
    label = file_name.split('_')[0]  # Предположим, что название класса — это первая часть имени файла
    df['label'] = label
    dfs.append(df)

# Объединение всех датафреймов в один
data = pd.concat(dfs, ignore_index=True)

# Обработка текста: объединение необходимых столбцов в один
data['text'] = data[['headline', 'description', 'content']].fillna('').agg(' '.join, axis=1)
data = data.drop(columns=['headline', 'description', 'content', 'url'], errors='ignore')

print(data.head())  # Просмотр первых строк для проверки


['education_data.csv', 'sports_data.csv', 'entertainment_data.csv', '.ipynb_checkpoints', 'business_data.csv', 'technology_data.csv', 'archive.zip']


IsADirectoryError: [Errno 21] Is a directory: '/content/data/.ipynb_checkpoints'

In [17]:
df = pd.read_csv("/content/archive.zip")

ValueError: Multiple files found in ZIP file. Only one file per ZIP: ['business_data.csv', 'education_data.csv', 'entertainment_data.csv', 'sports_data.csv', 'technology_data.csv']

In [ ]:
df.columns

Index(['headlines', 'description', 'content', 'url', 'category'], dtype='object')

In [ ]:
df['text'] = df['headlines'].fillna('') + ' ' + df['description'].fillna('') + ' ' + df['content'].fillna('')

In [ ]:
df = df.drop(columns=['headlines', 'description', 'content', 'url'], errors='ignore')

In [ ]:
df.head()

,category,text
0,business,Nirmala Sitharaman to equal Morarji Desai’s re...
1,business,"‘Will densify network, want to be at least no...."
2,business,Air India group to induct an aircraft every si...
3,business,Red Sea woes: Exporters seek increased credit ...
4,business,Air India group to induct a plane every 6 days...


In [ ]:
class MyDataset(Dataset):
    def __init__(self, texts, labels, max_len=100):
        self.texts = texts
        self.labels = labels
        self.max_len = max_len
        self.label_encoder = LabelEncoder().fit(labels)
        self.vocab = GloVe(name='6B', dim=100)  # Загружаем предобученные вектора GloVe

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        label = self.label_encoder.transform([self.labels.iloc[idx]])[0]
        label = torch.tensor(label, dtype=torch.long)  # Убираем .squeeze()

        text = self.texts.iloc[idx]
        tokens = text.split()

        if len(tokens) > self.max_len:
            tokens = tokens[:self.max_len]
        else:
            tokens += [''] * (self.max_len - len(tokens))

        X = self.vocab.get_vecs_by_tokens(tokens, lower_case_backup=True)
        return X, label

In [ ]:
dataset = MyDataset(df['text'], df['category'])

In [ ]:
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [ ]:
class SimpleTextClassifierWithAttention(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(SimpleTextClassifierWithAttention, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.relu = nn.ReLU()
        self.attention = nn.Linear(hidden_dim, 1)  # Attention layer
        self.flatten = nn.Flatten()  # Flatten layer
        self.fc2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)

        # Attention mechanism
        attention_weights = torch.softmax(self.attention(x), dim=1)  # Вычисляем веса внимания
        x = x * attention_weights  # Применяем веса внимания

        x = self.flatten(x)  # Flattening after attention

        x = self.fc2(x)
        return x


In [ ]:
input_dim = 100
hidden_dim = 128
output_dim = len(df['category'].unique())

model = SimpleTextClassifier(input_dim, hidden_dim, output_dim).to(device)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
def train_model(model, train_loader, criterion, optimizer, epochs=5):
    model.train()
    for epoch in range(epochs):
        total_loss = 0.0
        correct = 0
        total = 0
        for texts, labels in train_loader:
            texts, labels = texts.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(texts)

            # Преобразование меток в нужную форму [batch_size, 1]
            labels = labels.view(-1).reshape(-1, 1)

            # Теперь снова преобразуем метки в одномерный формат для функции потерь
            labels = labels.view(-1)

            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        print(f'Epoch {epoch+1}, Loss: {total_loss/len(train_loader):.4f}, Accuracy: {100 * correct / total:.2f}%')


In [ ]:
train_model(model, train_loader, criterion, optimizer, epochs=5)


RuntimeError: Expected target size [32, 1], got [32]

In [ ]:
def evaluate_model(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for texts, labels in test_loader:
            texts, labels = texts.to(device), labels.to(device)
            outputs = model(texts)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print(f'Accuracy: {100 * correct / total:.2f}%')

evaluate_model(model, test_loader)


In [ ]:
evaluate_model(model, test_loader)

In [ ]:
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import Dataset, DataLoader
from torchtext.vocab import GloVe
import torch.nn as nn
import torch.optim as optim

# Устройство для выполнения вычислений
device = "cuda" if torch.cuda.is_available() else "cpu"

# Чтение и подготовка данных
df = pd.read_csv("/content/business_data.csv.zip")

# Объединение столбцов 'headlines', 'description' и 'content' в один текстовый столбец
df['text'] = df['headlines'].fillna('') + ' ' + df['description'].fillna('') + ' ' + df['content'].fillna('')

# Удаление ненужных столбцов
df = df.drop(columns=['headlines', 'description', 'content', 'url'], errors='ignore')

# Определение класса Dataset
class MyDataset(Dataset):
    def __init__(self, texts, labels, max_len=100):
        self.texts = texts
        self.labels = labels
        self.max_len = max_len
        self.label_encoder = LabelEncoder().fit(labels)
        self.vocab = GloVe(name='6B', dim=100)  # Загружаем предобученные вектора GloVe

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        label = self.label_encoder.transform([self.labels.iloc[idx]])[0]
        label = torch.tensor(label, dtype=torch.long)  # Метки в формате long

        text = self.texts.iloc[idx]
        tokens = text.split()

        if len(tokens) > self.max_len:
            tokens = tokens[:self.max_len]
        else:
            tokens += [''] * (self.max_len - len(tokens))

        X = self.vocab.get_vecs_by_tokens(tokens, lower_case_backup=True)
        return X, label

# Подготовка данных для обучения
dataset = MyDataset(df['text'], df['category'])

train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Определение модели с Attention
class SimpleTextClassifierWithAttention(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(SimpleTextClassifierWithAttention, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.relu = nn.ReLU()
        self.attention = nn.Linear(hidden_dim, 1)  # Attention layer
        self.fc2 = nn.Linear(hidden_dim, output_dim)  # Линейный слой для классификации

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)

        # Attention mechanism
        attention_weights = torch.softmax(self.attention(x), dim=1)  # Вычисляем веса внимания
        x = x * attention_weights  # Применяем веса внимания

        # Суммируем по оси слов для получения фиксированного представления
        x = torch.sum(x, dim=1)

        x = self.fc2(x)
        return x

# Настройка параметров и создание экземпляра модели
input_dim = 100  # Размерность входа (соответствует размерности GloVe)
hidden_dim = 128  # Количество нейронов в скрытом слое
output_dim = len(df['category'].unique())  # Количество категорий

model = SimpleTextClassifierWithAttention(input_dim, hidden_dim, output_dim).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Определение функции обучения
def train_model(model, train_loader, criterion, optimizer, epochs=5):
    model.train()
    for epoch in range(epochs):
        total_loss = 0.0
        correct = 0
        total = 0
        for texts, labels in train_loader:
            texts, labels = texts.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(texts)

            # Преобразование меток в одномерный формат для функции потерь
            labels = labels.view(-1)  # Удаляем лишнее измерение

            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        print(f'Epoch {epoch+1}, Loss: {total_loss/len(train_loader):.4f}, Accuracy: {100 * correct / total:.2f}%')

# Запуск обучения модели
train_model(model, train_loader, criterion, optimizer, epochs=5)

# Определение функции оценки модели
def evaluate_model(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for texts, labels in test_loader:
            texts, labels = texts.to(device), labels.to(device)
            outputs = model(texts)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print(f'Accuracy: {100 * correct / total:.2f}%')

# Оценка модели
evaluate_model(model, test_loader)


Epoch 1, Loss: 0.0000, Accuracy: 100.00%
Epoch 2, Loss: 0.0000, Accuracy: 100.00%
Epoch 3, Loss: 0.0000, Accuracy: 100.00%
Epoch 4, Loss: 0.0000, Accuracy: 100.00%
Epoch 5, Loss: 0.0000, Accuracy: 100.00%
Accuracy: 100.00%
